In [1]:
#<img src="Figs/GEOS_logo.pdf" width="500"/>

# generate Synthetic InSAR (asd & dsd) and GNSS: <font color=blue>"generate_synthetic_data.ipynb"</font>
#### Dec 09, 2021  <font color=red>(v. testing)</font>
##### Jeonghyeop Kim (jeonghyeop.kim@gmail.com)


**input files :**  
> **`losx_DT173_regular_lat_long.dat`**, **`losy_DT173_regular_lat_long.dat`**, **`losz_DT173_regular_lat_long.dat`** \
> **`losx_AT64_regular_lat_long.dat`**, **`losy_AT64_regular_lat_long.dat`**, **`losz_AT64_regular_lat_long.dat`** \
> **`vel_horizontal_from_vertical_displacement.gmt`**, **`vel_UCERF3_corner.gmt`**, **`vel_vertical.gmt`** \
> **`coordinate_list_full.dat`** : GNSS stations

**output files :** 
> **`dLOS_DT173_synthetic.dat`** & **`dLOS_AT64_synthetic.dat`** & **`GNSS_horizontal_synthetic.gmt`** \

0. This code is a part of the joint inversion project (project4: joint inversion of GNSS and InSAR)
1. It generates Synthetic InSAR data sets (Ascending & Descending) and GNSS data sets
2. The synthetic InSAR data sets will be sampled in a regular long & lat grid
3. The synthetic horizontal GNSS data will be sampled in the NOTA (PBO) stations. 

In [2]:
import numpy as np
import pandas as pd
from distance_calculation import distance_calculation

In [3]:
scaling_factor = 0.3 #for vertical

In [4]:
error_level_InSAR = 0.3 # 30 %
error_level_GNSS = 0.05 # 5 %

In [5]:
data_losxD=np.loadtxt("losx_DT173_regular_lat_long.dat")
data_losyD=np.loadtxt("losy_DT173_regular_lat_long.dat")
data_loszD=np.loadtxt("losz_DT173_regular_lat_long.dat")

data_losxA=np.loadtxt("losx_AT64_regular_lat_long.dat")
data_losyA=np.loadtxt("losy_AT64_regular_lat_long.dat")
data_loszA=np.loadtxt("losz_AT64_regular_lat_long.dat")

data_hori_from_vert=np.loadtxt("vel_horizontal_from_vertical_displacement.gmt")
data_hori=np.loadtxt("vel_UCERF3_corner.gmt")
data_vert=np.loadtxt("vel_vertical.gmt")

data_GNSS_coor = np.loadtxt("coordinate_list_full.dat")
data_GNSS_coor[:,0]=data_GNSS_coor[:,0]-360
data_GNSS_coor = data_GNSS_coor[(data_GNSS_coor[:,0]>=-117),:]
data_GNSS_coor = data_GNSS_coor[(data_GNSS_coor[:,0]<=-116),:]
data_GNSS_coor = data_GNSS_coor[(data_GNSS_coor[:,1]>=33.2),:]
data_GNSS_coor = data_GNSS_coor[(data_GNSS_coor[:,1]<=34.1),:]

lon = data_hori[:,0]
lat = data_hori[:,1]
zeroVec = data_hori[:,5]

hori_UCERF_x = data_hori[:,2]
hori_vert_x = data_hori_from_vert[:,2]
u_x = hori_UCERF_x + hori_vert_x*scaling_factor

hori_UCERF_y = data_hori[:,3]
hori_vert_y = data_hori_from_vert[:,3]
u_y = hori_UCERF_y + hori_vert_y*scaling_factor

u_z = data_vert[:,2]*scaling_factor

losxD=data_losxD[:,2]
losyD=data_losyD[:,2]
loszD=data_loszD[:,2]

losxA=data_losxA[:,2]
losyA=data_losyA[:,2]
loszA=data_loszA[:,2]



In [6]:
#generate synthetic data : InSAR
dLOS_D = u_x*losxD + u_y*losyD + u_z*loszD
dLOS_A = u_x*losxA + u_y*losyA + u_z*loszA

In [7]:
GNSS_x = np.zeros((len(data_GNSS_coor),))
GNSS_y = np.zeros((len(data_GNSS_coor),))
# GNSS_sx = np.zeros((len(data_GNSS_coor),))
# GNSS_sy = np.zeros((len(data_GNSS_coor),))
GNSS_corrxy = np.zeros((len(data_GNSS_coor),))

for i in range(len(data_GNSS_coor)):
    lon_GNSS = data_GNSS_coor[i,0]
    lat_GNSS = data_GNSS_coor[i,1]
    
    idx_total=np.argwhere((lon<=lon_GNSS+0.004) & (lon>=lon_GNSS-0.004))
    #print(lon_GNSS, lat_GNSS)
    #print(len(idx))
    
    lon_search = lon[idx_total]
    lat_search = lat[idx_total]
    
    dist = np.zeros((len(lon_search),1))
    
    for j in range(len(lon_search)):
        
        lon_candidate = lon_search[j,0]
        lat_candidate = lat_search[j,0]
        #print(lon_search,lat_candidate)
        
        distance, az = distance_calculation(lat_GNSS,lon_GNSS,lat_candidate,lon_candidate)
        dist[j,0] = distance
    
    index_search=np.argmin(dist)
    
    idx_final = idx_total[index_search]
    
    GNSS_x[i]=u_x[idx_final]
    GNSS_y[i]=u_y[idx_final]
    

In [8]:
GNSS_lon=data_GNSS_coor[:,0]
GNSS_lat=data_GNSS_coor[:,1]

In [9]:
random_GNSSx = error_level_GNSS * np.random.uniform(-1,1,len(GNSS_x))
GNSS_x = GNSS_x + GNSS_x * random_GNSSx
GNSS_sx = GNSS_x * error_level_GNSS

random_GNSSy = error_level_GNSS * np.random.uniform(-1,1,len(GNSS_y))
GNSS_y = GNSS_y + GNSS_y * random_GNSSy
GNSS_sy = GNSS_y * error_level_GNSS

random_InSAR_D = error_level_InSAR * np.random.uniform(-1,1,len(dLOS_D))
dLOS_D = dLOS_D + dLOS_D * random_InSAR_D

random_InSAR_A = error_level_InSAR * np.random.uniform(-1,1,len(dLOS_A))
dLOS_A = dLOS_A + dLOS_A * random_InSAR_A

In [10]:
d = {'lon': GNSS_lon, 'lat': GNSS_lat, 've': GNSS_x, 'vn': GNSS_y, 'se': GNSS_sx, 'sn': GNSS_sy, 'corr': GNSS_corrxy}
df = pd.DataFrame(d)

In [11]:
df.to_csv("GNSS_horizontal_synthetic.gmt",header=None,index=None,sep=' ')

In [12]:
d_D = {'lon': lon, 'lat': lat, 'dLOS': dLOS_D}
df_D = pd.DataFrame(d_D)
df_D.to_csv("dLOS_DT173_synthetic.dat",header=None,index=None,sep=' ')


d_A = {'lon': lon, 'lat': lat, 'dLOS': dLOS_A}
df_A = pd.DataFrame(d_A)
df_A.to_csv("dLOS_AT64_synthetic.dat",header=None,index=None,sep=' ')
